In [32]:
import numpy as np
import tensorflow as tf
from os.path import  join
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten, Dense, Activation
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import f1_score

In [43]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=5,
                           n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [57]:
# Preprocess the data
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [58]:
BATCH_SIZE = 32

In [59]:
model = Sequential()
model.add(Input(shape=(20,)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [60]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [73]:
class LossHistory(tf.keras.callbacks.Callback):
    def __init__(self,validation_data):
        self.x_test = validation_data[0]
        self.y_test= validation_data[1]

    def on_train_begin(self, logs={}):
        self.history={'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': [],'val_f1': []}

    def on_epoch_end(self, epoch, logs={}):
        self.history['loss'].append(logs.get('loss'))
        self.history['accuracy'].append(logs.get('accuracy'))

        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history['val_accuracy'].append(logs.get('val_accuracy'))

        y_pred= self.model.predict(self.x_test)
        y_label_pred=np.argmax(y_pred,axis=1)
        y_test_arg = self.y_test

        print(self.y_test.shape)
        print(y_label_pred.shape)

        F1 = f1_score(y_test_arg, y_label_pred, average='micro')
        self.history['val_f1'].append(F1)
        print(' Val F1 Score : ', np.round(F1,4))

In [74]:
history_callback = LossHistory(validation_data=([X_test, y_test]))

In [75]:
history_1 = model.fit(x=X_train,\
                        y=y_train,\
                        batch_size=BATCH_SIZE,\
                        epochs=10,\
                        steps_per_epoch = int(np.ceil(X_train.shape[0] / BATCH_SIZE) ),\
                        validation_data=(X_test, y_test),
                        callbacks=history_callback)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
(250, 2)
(250,)


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets